<a href="https://colab.research.google.com/github/reeaarana/Advanced-ML-Projects/blob/main/00_setup_and_smoke_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install pytorch

In [ ]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
import os
print("If this was run in Colab with a GPU runtime but you installed the cpu wheel, your code will run on CPU.")


Looking in indexes: https://download.pytorch.org/whl/cpu
If this was run in Colab with a GPU runtime but you installed the cpu wheel, your code will run on CPU.


verification cell

In [ ]:
import sys, torch
print("Python:", sys.version.splitlines()[0])
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print("CUDA version seen by torch:", torch.version.cuda)
        print("CUDA device name:", torch.cuda.get_device_name(0))
    except Exception as e:
        print("GPU detected but error reading details:", e)


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Torch: 2.8.0+cu126
CUDA available: True
CUDA version seen by torch: 12.6
CUDA device name: NVIDIA A100-SXM4-80GB


install requiered packages (best-effort)

In [ ]:
# Run in Colab cell. This installs common packages; torch/geometric may need special handling.
!pip install --upgrade pip
!pip install nibabel nilearn scikit-image pandas umap-learn captum

installingi PyG

In [ ]:
# Example (may fail if the exact wheels are not available)
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.8.0+cu126.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.8.0+cu126.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.8.0+cu126.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.8.0+cu126.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 77.8 MB/s  0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 22.7 MB/s  0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 109.3 MB/s  0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.8.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s  0:00:00
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


mount google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# optional convenience symlink to use /MyDrive
import os
if not os.path.exists('/MyDrive'):
    try:
        os.symlink('/content/drive/MyDrive', '/MyDrive')
    except Exception:
        pass
print("Drive available at /MyDrive (if symlink created) and /content/drive/MyDrive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive available at /MyDrive (if symlink created) and /content/drive/MyDrive


create project folders on drive

In [ ]:
import os
project_root = '/MyDrive/oasis_project'
subdirs = ['notebooks', 'data', 'outputs', 'logs']
for d in subdirs:
    path = os.path.join(project_root, d)
    os.makedirs(path, exist_ok=True)
    print('Ensured', path)


Ensured /MyDrive/oasis_project/notebooks
Ensured /MyDrive/oasis_project/data
Ensured /MyDrive/oasis_project/outputs
Ensured /MyDrive/oasis_project/logs


env + gpu check

In [ ]:
import sys, pkg_resources, torch
print("Python:", sys.version.splitlines()[0])
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print("CUDA device:", torch.cuda.get_device_name(0))
    except Exception:
        pass

# print versions of key packages if installed
for p in ['nibabel','nilearn','pandas','umap-learn','captum','torch_geometric']:
    try:
        print(p, "->", pkg_resources.get_distribution(p).version)
    except Exception:
        print(p, "-> not installed")


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Torch: 2.8.0+cu126
CUDA available: True
CUDA device: NVIDIA A100-SXM4-80GB
nibabel -> 5.3.2
nilearn -> 0.12.1
pandas -> 2.2.2
umap-learn -> 0.5.9.post2
captum -> 0.8.0
torch_geometric -> 2.6.1


/tmp/ipython-input-2597987680.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import sys, pkg_resources, torch


Tiny GraphSage PyG smoke test

In [ ]:
# If torch_geometric imports succeed this runs a 1-epoch GraphSAGE on a 4-node fake graph
import torch
use_pyg = False
try:
    from torch_geometric.data import Data
    from torch_geometric.nn import SAGEConv
    use_pyg = True
except Exception as e:
    print("PyG not importable; skipping GraphSAGE:", e)

if use_pyg:
    import torch.nn.functional as F
    x = torch.randn((4, 3))
    edge_index = torch.tensor([[0,1,2,3,0,2],[1,0,3,2,2,0]], dtype=torch.long)
    y = torch.tensor([0,1,0,1], dtype=torch.long)
    data = Data(x=x, edge_index=edge_index, y=y)

    class TinySAGE(torch.nn.Module):
        def __init__(self, in_ch, hid_ch, out_ch):
            super().__init__()
            self.conv1 = SAGEConv(in_ch, hid_ch)
            self.conv2 = SAGEConv(hid_ch, out_ch)
        def forward(self, x, edge_index):
            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = self.conv2(x, edge_index)
            return x

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = TinySAGE(3, 8, 2).to(device)
    data = data.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = torch.nn.CrossEntropyLoss()
    model.train()
    opt.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_fn(out, data.y)
    loss.backward()
    opt.step()
    print("GraphSAGE epoch 0 loss:", loss.item())


GraphSAGE epoch 0 loss: 0.8971723318099976


pure Py-Torch fallback smoke test

In [ ]:
# Run when PyG isn't available. Tiny MLP on random data for 1 epoch.
import torch, torch.nn as nn, torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.randn((16, 10), device=device)
y = (X.sum(dim=1) > 0).long()
model = nn.Sequential(nn.Linear(10,32), nn.ReLU(), nn.Linear(32,2)).to(device)
opt = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
opt.zero_grad()
out = model(X)
loss = loss_fn(out, y)
loss.backward()
opt.step()
print("Pure-Torch epoch 0 loss:", loss.item())


Pure-Torch epoch 0 loss: 0.7792071104049683
